In [1735]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from itertools import combinations

In [1736]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1587,tt0356634,50000000,200804534,Garfield,Bill Murray|Breckin Meyer|Jennifer Love Hewitt...,Peter Hewitt,Get ready for frisky business.,"Garfield, the fat, lazy, lasagna lover, has ev...",80,Animation|Comedy|Family,Twentieth Century Fox Film Corporation|Davis E...,6/10/2004,5.2,2004
1390,tt0384680,20000000,12482775,The Weather Man,Nicolas Cage|Michael Caine|Hope Davis|Gemmenne...,Gore Verbinski,Dave Spritz is about to take his best shot . ....,"A Chicago weather man, separated from his wife...",101,Comedy|Drama,Paramount Pictures|Escape Artists|Kumar Mobili...,10/20/2005,5.7,2005
1650,tt0377062,45000000,21009180,Flight of the Phoenix,Dennis Quaid|Tyrese Gibson|Giovanni Ribisi|Mir...,John Moore,"Out of the ashes, hope will rise.",When an Amacor oil rig in the Gobi Desert of M...,113,Action|Adventure|Drama|Thriller,Twentieth Century Fox Film Corporation|Davis E...,12/17/2004,5.4,2004
896,tt0253754,60000000,67312826,Star Trek: Nemesis,Patrick Stewart|Jonathan Frakes|LeVar Burton|B...,Stuart Baird,A generation's final journey... begins.,En route to the honeymoon of William Riker to ...,117,Science Fiction|Action|Adventure|Thriller,Paramount Pictures,12/12/2002,6.1,2002
276,tt0448011,50000000,155446362,Knowing,Nicolas Cage|Rose Byrne|Chandler Canterbury|Be...,Alex Proyas,Knowing is everything...,A teacher opens a time capsule that has been d...,121,Action|Adventure|Drama|Mystery|Science Fiction,Summit Entertainment|Mystery Clock Cinema|Esca...,3/19/2009,5.7,2009


In [1737]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [1738]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

def preparation_answer(incomming_data):
    ''' Функция для подготовки ответа в формате платформы.
        Формирует выходной результат из имени фильма и его идентификатора'''
    
    # берем первую строку дата фрейма и определяем имя фильма
    movie_name = incomming_data.original_title.iloc[0]
    
    #  определяем id фильма    
    movie_id = incomming_data.imdb_id.iloc[0]
    print(movie_name, ' (', movie_id , ')', sep='')
    return



def year_detect(release_date):
    ''' Функция получения года из четырех цифр, идущих подряд.
        Если год не найден во входной строке, будет возвращено 1970'''
    
    # регулярным выражением проверяем в строке 4 цифры подряд
    result = re.search(r'\d{4}', release_date)
   
    # если найдено
    if result:
        # получаем найденную подстроку и приводим к типу - целое
        return int(result.group(0))
    else:
        # вернем это значение если не смогли найти по шаблону ничего
        return 1970


def month_detect(release_date):
    ''' Функциия получения месяца из первых двух или одно цифры, идущих в начале строки.
        Если цифры не найдены в начале строки - вернет 0. Если полученный номер месяца вне диапазона - вернет 0'''
    
    # регулярным выражением проверяем в строке 2 цифры подряд или 1 цифру в начале строки
    result = re.search(r'^\d\d?', release_date)
    
    # если совпадение есть с шаблоном регулярного выражения
    if result:
        # получаем найденную подстроку и приводим к типу - целое
        month = int(result.group(0))
        # проверяем что значение месяца в валидном диапазоне, иначе возвращем - 0
        if month < 1 or month > 12:
            month = 0
        return month
    else:
        return 0


def series_aggregation_first_column(incoming_data, agg_type='sum', top=5, sort_direction=False,):
    ''' Функция на вход  получвает серию, далее разделяя индексы по символу "|" подсчитывает сумму значений
        Выходной результат - ранжированная серия, ограниченная по длине'''
    
    # создаем пустую серию для выходного результата 
    result_series = pd.Series(dtype=object)
    
    # проходим циклом по парам - индекс и значение 
    for column1, column2 in incoming_data.items():
        # если вид агрегации накопительный, то будем суммировать. Иначе - считать кол-во
        if agg_type == 'sum':
            agg_parametr = column2
        else:
            agg_parametr = 1
        # если индес составной - разделяем на отдельные занчения    
        for item in column1.split('|'):
            # собираем данные в серию, где индексом будут некие ключевые параметры
            # а значениями - сумма по этим параметрам
            if item in result_series.index:
                result_series[item] += agg_parametr
            else:
                result_series[item] = agg_parametr
    # возвращаем итоговую серию данных
    return result_series.sort_values(ascending=sort_direction).head(top)
  

# создаем и заполняем новую колонку "Прибыльность"
data['profit'] = data.revenue - data.budget

# создаем и заполняем новую колонку "год выпуска фильма"
data['release_year'] = data.release_date.apply(year_detect)

# создаем и заполняем новую колонку "месяц выпуска фильма"
data['release_month'] = data.release_date.apply(month_detect)

# создаем и заполняем новую колонку "количество слов в обзоре фильма"
# функция поиска всех вхождений слов возвращает массив совпадений, его длина и есть количество слов
# словом считает все от одного символа и длинее
data['word_count_overview'] = data.overview.apply(lambda s: len(re.findall(r'\w+', s.lower())))


In [1739]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,word_count_overview
292,tt1148204,20000000,41596251,Orphan,Vera Farmiga|Peter Sarsgaard|Isabelle Fuhrman|...,Jaume Collet-Serra,Can you keep a secret?,A married couple with a rocky past adopt 9-yea...,123,Horror|Thriller|Mystery,Dark Castle Entertainment|Studio Babelsberg St...,7/24/2009,6.5,2009,21596251,7,30
1414,tt0424774,50000000,69425966,The Adventures of Sharkboy and Lavagirl,Taylor Lautner|Taylor Dooley|Cayden Boyd|Georg...,Robert Rodriguez,Smaller heroes. Just as super.,Max conjures up his perfect dreamworld...the P...,92,Adventure|Fantasy|Action|Family,Columbia Pictures Corporation|Dimension Films,6/10/2005,4.3,2005,19425966,6,44
1094,tt0328828,55000000,231449203,American Wedding,Jason Biggs|Alyson Hannigan|Seann William Scot...,Jesse Dylan,Forever hold your piece.,"With high school a distant memory, Jim and Mic...",103,Comedy|Romance,Universal Pictures|Zide-Perry Productions|Live...,8/1/2003,6.0,2003,176449203,8,55
873,tt0181689,102000000,358372926,Minority Report,Tom Cruise|Colin Farrell|Samantha Morton|Max v...,Steven Spielberg,The system is perfect until it comes after you.,"John Anderton is a top ""Precrime"" cop in the l...",145,Action|Thriller|Science Fiction|Mystery,DreamWorks SKG|Cruise/Wagner Productions|Ambli...,6/20/2002,6.9,2002,256372926,6,52
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,10/16/2015,8.0,2015,29401758,10,49


In [1805]:
data.columns

Index(['imdb_id', 'budget', 'revenue', 'original_title', 'cast', 'director',
       'tagline', 'overview', 'runtime', 'genres', 'production_companies',
       'release_date', 'vote_average', 'release_year', 'profit',
       'release_month', 'word_count_overview'],
      dtype='object')

 ####   Column            
---  ------               
 0.   imdb_id              
 1.   budget               
 2.   revenue              
 3.   original_title       
 4.   cast -- мультиполе, разделитель "|"
 5.   director -- мультиполе, разделитель "|"
 6.   tagline              
 7.   overview             
 8.   runtime              
 9.   genres -- мультиполе, разделитель "|"
 10.  production_companies -- мультиполе, разделитель "|"
 11.  release_date         
 12.  vote_average         
 13.  release_year         
 14.  profit
 15.  release_month
 16.  word_count_overiew

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [1740]:
# получаем максимальный бюджет и фильтруем все фильмы с бюджетом равным максимальному
movie_name = data[data.budget == data.budget.max()]

# формируем и выдаем ответ в стиле платформы 
preparation_answer(movie_name)



Pirates of the Caribbean: On Stranger Tides (tt1298650)


In [1741]:
# +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [1742]:
# ранжируем колонку с бюджетами фильма, последнее значение будет максимальным
max_budget = data.budget.sort_values().tail(1)

# берем значение индекса у максмального значения и получаем название фильма
data.loc[max_budget.index[0]].original_title

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [1743]:
# получаем максимальную длительность и фильтруем все фильмы с длительностью равной максимальной
movie_name = data[data.runtime == data.runtime.max()]

preparation_answer(movie_name)

Gods and Generals (tt0279111)


In [1744]:
# +
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1745]:
# получаем минимальную длительность и фильтруем все фильмы с длительностью равной минимальной
movie_name = data[data.runtime == data.runtime.min()]

preparation_answer(movie_name)

Winnie the Pooh (tt1449283)


In [1746]:
# +
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [1747]:
# получаем среднее значение по столбцу "длительность" и округляем до целого
round(data.runtime.mean())

110

In [1748]:
# +
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [1749]:
# получаем медианное значение по столбцу "длительность" и округляем до целого
round(data.runtime.median())

107

In [1750]:
# +
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1751]:
# лучше код получения столбца profit вынести в Предобработку что в начале

# получаем максимальное значение из столбца "прибыль", получаем фильмы у которых прибыль максимальна
movie_name = data[data.profit == data.profit.max()]

preparation_answer(movie_name)

Avatar (tt0499549)


In [1752]:
# +
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [1753]:
# получаем минимальное значение из столбца "прибыль", получаем фильмы у которых прибыль равна минимальной
movie_name = data[data.profit == data.profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1754]:
# +
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1755]:
# фильтруем датасет, выводим только те, у которых в колонке "прибыль" больше 0, определяем длину по вертикали,
# это и будет количеством фильмов
len(data[data.profit > 0])

1478

In [1756]:
# +
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1757]:
# фильтруем по столбцу "год выпуска", забирая только 2008 год, и получаем максимальное значение у этой выборки
# далее получаем фильмы у которых встречается максимальное значение в этой выборке
movie_name = data[data.profit == data.query('release_year == 2008').profit.max()]

preparation_answer(movie_name)

The Dark Knight (tt0468569)


In [1758]:
# +
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [1759]:
# фльтруем выборку по диапазону года выпуска, у этой выборке определеяем минимальный доход от фильма
# далее в этой выборке оставляем только фильмы с доходом равным минимальному
movie_name = data[data.profit == data.query('2012 <= release_year <= 2014').profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1760]:
# +
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [1761]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# формируем серию где индексы - жанры, а значения - количество фильмов с таким жанром
# обрабатываем функцией, так как в поле жанр может быть несколько значений
series_aggregation_first_column(data.genres.value_counts(), 'sum', top=1)


Drama    782
dtype: int64

In [1762]:
# +
answers['11'] = 'Drama'

ВАРИАНТ 2

In [1763]:
# создаем пустую коллекцию
genre_counter = Counter()

# делаем выгрузку из дата фрейма и заполняем коллекцию значениями жанров
for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        genre_counter[genre] += genre_count

# получаем максимальное значения в словаре
max_count = max(genre_counter.values())

# ищем значение ключа, в котором находится жанр с максимальным количеством
for genre, item in genre_counter.items(): 
    if item == max_count: print(genre)


Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1764]:
# формируем серию из прибыльных фильмов, где индес - жанр, значение - количество фильмов
profitable_film_genre = data[data.profit > 0].groupby(['genres'])['profit'].count()

# обрабатываем функцией, так как в поле "жанр" может быть несколько значений
series_aggregation_first_column(profitable_film_genre, top=1)


Drama    560
dtype: int64

In [1765]:
# +
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [1766]:
# формируем серию - группируем по полю директор и суммируя кассовые сборы
director_revenue = data.groupby(['director'])['revenue'].sum()

# обрабатываем функцией, так как в поле "директор" может быть несколько значений
series_aggregation_first_column(director_revenue, top=1)

Peter Jackson    6490593685
dtype: int64

In [1767]:
# +
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1768]:
# получаем из датафрейма два столбца "директор" и "жанры", отфильтровав по столбцу "жанр"
# по наличию в поле 'Action'
director_genres = data[ ['director','genres'] ][data.genres.str.contains('Action', na=False)]

# формируем серию - группируем по полю директор и получаем количество по полю "жанр"
director_genres = director_genres.groupby(['director'])['genres'].count()

# обрабатываем функцией, так как в поле "директор " может быть несколько значений
series_aggregation_first_column(director_genres, 'sum', top=3)


Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
dtype: int64

In [1769]:
# +
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1770]:
# фильтруем датафрейм по 2012 году, группируем по полю "Актер" и считаем сумму кассовых сборов 
top_actor_2012 = data.query('release_year == 2012').groupby(['cast'])['revenue'].sum()

# обрабатываем функцией, так как в поле "актер" может быть несколько значений
series_aggregation_first_column( top_actor_2012 ,'sum', top=1)


Chris Hemsworth    2027450773
dtype: int64

In [1771]:
# +
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1772]:
# высокобюджетные - фильмы, где бюджет выше среднего по данной выборке
# фильтруем датафрейм и группируем по полю "актер", считаю сумму кассовых сборов 
top_actor = data[data.budget > data.budget.mean()].groupby(['cast'])['revenue'].count()

# обрабатываем функцией, так как в поле "актер" может быть несколько значений
series_aggregation_first_column(top_actor ,'sum', top=3)


Matt Damon        18
Adam Sandler      17
Angelina Jolie    16
dtype: int64

In [1773]:
# +
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1774]:
# формируем датафрейм - оставляем строки где в поле "актер" есть упоминание о 'Nicolas Cage'
# проводим группировку данных по колонке "Жанр" и считаем количество записей для каждого жанра
favorite_genre = data[data.cast.str.contains('Nicolas Cage', na=False)].groupby(['genres'])['cast'].count()

# обрабатываем функцией, так как в поле "жанр" может быть несколько значений
series_aggregation_first_column( favorite_genre ,'sum', top=3)

Action      17
Thriller    15
Drama       12
dtype: int64

In [1775]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [1776]:
# отфильтровываем те записи, где кинокомания 'Paramount Pictures' и получаем минимальное значение
movie_name = data[data.profit == data[data.production_companies.str.contains('Paramount Pictures')].profit.min()]

preparation_answer(movie_name)

K-19: The Widowmaker (tt0267626)


In [1777]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1778]:
# группируем по году и для каждого считаем сумму кассовых сборов
# после ранжирования максимальное значение будет последним, поэтому берем 1 запись "с конца" серии данных
data.groupby(['release_year'])['revenue'].sum().sort_values().tail(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

In [1779]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1780]:
# фильтруем дата фрейм, оставляем данные где упоминается кинокомпания 'Warner Bros'
# результат сохраняем в новый датафрейм
wb_profit = data[data.production_companies.str.contains('Warner Bros', na=False)]

# группируем по году выпуска и считаем сумму по каждому году
# после ранжирования максимальное значение будет последним, поэтому берем 1 запись "с конца" серии данных
wb_profit.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(1)


release_year
2014    2295464519
Name: profit, dtype: int64

In [1781]:
# +
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1782]:
# Вариант 1

# получаем серию данных с количеством значений по каждому году
# ранжируем, последнее значение - максимальное
# вывод результата - номер месяца и количество
data.release_month.value_counts().sort_values().tail(1)

9    227
Name: release_month, dtype: int64

In [1783]:
# Вариант 2

# группируем году выпуска и считаем количество по каждому году
# ранжируем, последнее значение - максимальное
# вывод результата - номер месяца и количество
data.groupby(['release_month'])['imdb_id'].count().sort_values().tail(1)

release_month
9    227
Name: imdb_id, dtype: int64

In [1784]:
# +
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1785]:
# фильтруем датафрейм по номерам летних месяце, функция длины возвращает количество строк в датафрейме
# вывод результата - количество фильмов
len(data.query('release_month in [6, 7, 8]'))

450

In [1786]:
# +
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [1787]:
# фильтруем по номерам зимних месяцев
# формируем серию данных - "режиссеры" и "количество"
winter_director = data.query('release_month in [1, 2, 12]').director.value_counts()

# обрабатываем функцией, так как в поле "директор" может быть несколько значений
series_aggregation_first_column(winter_director,'sum', top=3)

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
dtype: int64

In [1788]:
# +
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1789]:
# определяем максимальную длину имени фильма
max_len = data.original_title.str.len().max()

# отфильтровываем датафрейм - оставляем только с максимальной длиной названия
# обращаемся к колонке "кинокомпания", разделяем по сплитеру и получаем значение кинокомпании
data[data.original_title.str.len() == max_len].production_companies.iloc[0].split('|')[1]


'Four By Two Productions'

In [1790]:
# +
answers['24'] = 'Four By Two Productions' 

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1791]:
# определяем максимальную длину описания фильма в количестве слов
word_count_overview_max = data[data.word_count_overview > data.word_count_overview.mean()].word_count_overview.max()

# отфильтровываем датафрейм - оставляем только с максимальным количеством слов в описании фильма
# обращаемся к колонке "кинокомпания", разделяем по сплитеру и получаем значение кинокомпании
data[data.word_count_overview == word_count_overview_max].production_companies.iloc[0].split('|')[3]


'Midnight Picture Show'

In [1792]:
# +
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [1793]:
# определяем значение 1% количества фильмов от общего числа фильмов в датасете
count_last_procent = round(0.01 * len(data))

# столбец "ср.оценка" сортируем по убыванию, забираем с начала серии данных количество индексов
# равное 1-му % 
last_procent = data.vote_average.sort_values(ascending=False).head(count_last_procent).index

# фильтруем датасет полученными индексами, сокращаем вывод столбцов до "название" и "ср.оценка"
top_movies = data.loc[last_procent][['original_title','vote_average']]
display(top_movies)

,original_title,vote_average
599,The Dark Knight,8.1
125,The Imitation Game,8.0
9,Inside Out,8.0
118,Interstellar,8.0
34,Room,8.0
1081,The Lord of the Rings: The Return of the King,7.9
138,The Grand Budapest Hotel,7.9
1183,The Wolf of Wall Street,7.9
370,Inception,7.9
119,Guardians of the Galaxy,7.9


In [1794]:
# Проверим, что из варианта №1 все 3 фильма находятся в в выборке 1% лучших фильмов

answer = ['Inside Out', 'The Dark Knight', '12 Years a Slave']

top_movies.query('original_title in @answer')


,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
1191,12 Years a Slave,7.9


In [1795]:
# +
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1796]:
# инициируем словарь счетчика
actors_pair = Counter()

# получаем все уникальные строки с актерами где есть хотя бы 1 разделитель
# фильмы с одним актером не содержат разделителя и не имеет ценности при расчете колобораций
actors = data[data.cast.str.contains('|')].cast.unique()

# запускаем циклической перебор по списку актеров, замеченных в совместной работе
for pair in actors:
    # формирем парные комбинации, предварительно сортируем разделенные значения, 
    # чтобы исключить создание нового ключа для поменявшихся местами пары имен актеров
    for actor in combinations(sorted(pair.split('|')), 2):
        # формируем ключевое поле словаря - пара из имен актеров через знак разделителя
        pair_key = actor[0] + ' & ' + actor[1]
        # заносим в словарь сформированную пару актеров
        actors_pair[pair_key] += 1

# определяем максимальное количество совместных съемок
count_colloboration_max = max(actors_pair.values())

# выведем топ 3 верхних значений по количеству совместных съемок
print(sorted(actors_pair.values(), reverse=True)[:3])

# выведем из словаря только пары актеров, с максимальным количеством совместных съемок
for pair, count_colloboration in actors_pair.items():
    if count_colloboration == count_colloboration_max: 
        print(pair)


[8, 8, 8]
Daniel Radcliffe & Emma Watson
Daniel Radcliffe & Rupert Grint
Emma Watson & Rupert Grint


In [1797]:
# +
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

In [1798]:
pair_actors = [  'Johnny Depp & Helena Bonham Carter',
                'Ben Stiller & Owen Wilson',
                'Vin Diesel & Paul Walker',
                'Adam Sandler & Kevin James',
                'Daniel Radcliffe & Rupert Grint']

actor_pair_work = pd.Series(dtype=object)

for pair_actor in pair_actors:
    actor1, actor2 =  pair_actor.split(' & ')
    pair_work_count = data[data.cast.str.contains(actor1) & data.cast.str.contains(actor2)].imdb_id.count()
    #print(pair_work_count, '|', pair_actor)
    actor_pair_work[pair_actor] = pair_work_count

actor_pair_work.sort_values(ascending=False).head(1).index[0]


'Daniel Radcliffe & Rupert Grint'

# Submission

In [1799]:
# в конце можно посмотреть свои ответы к каждому вопросу

# Вопрос № 1 ........ [ + ]
# Вопрос № 2 ........ [ + ]
# Вопрос № 3 ........ [ + ]
# Вопрос № 4 ........ [ + ]
# Вопрос № 5 ........ [ + ]
# Вопрос № 6 ........ [ + ]
# Вопрос № 7 ........ [ + ]
# Вопрос № 8 ........ [ + ]
# Вопрос № 9 ........ [ + ]
# Вопрос № 10 ....... [ + ]
# Вопрос № 11 ....... [ + ]
# Вопрос № 12 ....... [ + ]
# Вопрос № 13 ....... [ + ]
# Вопрос № 14 ....... [ + ]
# Вопрос № 15 ....... [ + ]
# Вопрос № 16 ....... [ + ]
# Вопрос № 17 ....... [ + ]
# Вопрос № 18 ....... [ + ]
# Вопрос № 19 ....... [ + ]
# Вопрос № 20 ....... [ + ]
# Вопрос № 21 ....... [ + ]
# Вопрос № 22 ....... [ + ]
# Вопрос № 23 ....... [ + ]
# Вопрос № 24 ....... [ + ]
# Вопрос № 25 ....... [ + ]
# Вопрос № 26 ....... [ + ]
# Вопрос № 27 ....... [ + ]

answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [1800]:
# и убедиться что ни чего не пропустил)
len(answers)

27

### Спасибо за внимание!